In [1]:
import pickle
import pandas as pd
import numpy as np
from ml_assemblr.main_components.data_pod import DataPod
from ml_assemblr.main_components.transformer import Serializer
from home_credit_helper.constant import *

In [2]:
from ml_assemblr.transfromers.value_replacer import ValueReplacer
from ml_assemblr.transfromers.column_devider import ColumnDivider

In [3]:
pd.set_option("display.max_columns", None)

In [ ]:
with open(".cache/01_dp.pkl", "rb") as f:
    dp: DataPod = pickle.load(f)

# 1. Application

In [ ]:
dp.dfs[APPLICATIONS].head()

In [ ]:
# table_name = APPLICATIONS
replacing_mappers = {"DAYS_EMPLOYED": {365243: np.nan}}
value_replacer = ValueReplacer(replacing_mappers=replacing_mappers)
dp = dp.fit_transform(value_replacer)

In [ ]:
divider_configs = [
    ("DAYS_EMPLOYED_PERC", "DAYS_EMPLOYED", "DAYS_BIRTH"),
    ("INCOME_CREDIT_PERC", "AMT_INCOME_TOTAL", "AMT_CREDIT"),
    ("INCOME_PER_PERSON", "AMT_INCOME_TOTAL", "CNT_FAM_MEMBERS"),
    ("ANNUITY_INCOME_PERC", "AMT_ANNUITY", "AMT_INCOME_TOTAL"),
    ("PAYMENT_RATE", "AMT_ANNUITY", "AMT_CREDIT"),
]
divider_pipeline = []
for target, numerator, denominator in divider_configs:
    divider_pipeline.append(
        ColumnDivider(
            target_col_name=target, numerator_col_name=numerator, denominator_col_name=denominator
        )
    )
divider_pipeline = Serializer(transformers=divider_pipeline)
    
dp = dp.fit_transform(divider_pipeline)

In [ ]:
pipeline = Serializer(transformers=[divider_pipeline, value_replacer])

In [ ]:
dp = dp.fit_transform(pipeline)

In [ ]:
raise

In [ ]:
from ml_assemblr.transfromers.encoder.one_hot_encoder import OneHotEncoder

In [ ]:
df = dp.main_df
categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and col != "split"]

In [ ]:
categorical_columns[:5]

In [ ]:
df["NAME_CONTRACT_TYPE"].head()

In [ ]:
one_hot_encoder = OneHotEncoder(target_df_name=APPLICATIONS, categorical_col_names=categorical_columns)

In [ ]:
print(dp.main_df.shape)
dp.main_df.head()

In [ ]:
dp = dp.fit_transform(one_hot_encoder)

In [ ]:
print(dp.main_df.shape)
dp.main_df.head()

In [ ]:
display

In [ ]:
dp.footprints[-1].one_hot_encoded_feature_names

In [ ]:
raise

In [ ]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [ ]:
df = dp.main_df

In [ ]:
df["ORGANIZATION_TYPE"]

In [ ]:
df2, cat_cols = one_hot_encoder(df, nan_as_category=False)

In [ ]:
df2["NAME_CONTRACT_TYPE_Cash loans"]

In [ ]:
df["ORGANIZATION_TYPE"]

In [ ]:
df

In [ ]:
import re

# s = "NAME_CONTRACT_TYPE_Cash loans"
s = "NAME_TYPE_SUITE_Spouse, ,  partner_Co-Op"

# Use regex to remove spaces and convert to uppercase
s = re.sub(r"[^a-zA-Z0-9]+", "_", s).upper()

print(s)

In [ ]:
cat_cols

In [ ]:
df["split_test"]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Example data
data = [['Male', 'New York'],
        ['Female', 'Chicago'],
        ['Male', 'Los Angeles']]

# Create DataFrame
df = pd.DataFrame(data, columns=['Gender', 'City'])

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform data
encoded_data = encoder.fit_transform(df[['Gender', 'City']])

# Convert sparse matrix to DataFrame
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['Gender', 'City']))

print(encoded_df)

In [ ]:
encoded_data.toarray()

In [ ]:
encoder.get_feature_names_out()